# Exercise 2: Testing of various longwave radiation parameterizations

In this excercise you will explore how different parameterizations of longwave radiation affects the modeled snowpack. We will explore this by using the different available parameterizations in MetSim.

To begin with, we will import everything we need. You are also provided with an initial MetSim configuration which is the same as the one from the previous exercise, though formatted for brevity. Run these two cells to get started.

In [ ]:
# modules 
import os
import pysumma as ps
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from metsim import MetSim

In [ ]:
config = {
    # Input files
    "domain": './data/reynolds/forcing_daily.nc',
    "forcing": './data/reynolds/forcing_daily.nc',
    "state": './data/reynolds/forcing_daily.nc',
    # Output location/naming
    "out_dir": './data/reynolds/',
    "out_prefix": 'forcing_metsim_uniform',
    # Run configuration/parameters
    "start": "2005/06/01",
    "stop": "2006/10/01",
    "time_step": 60,
    "period_ending": True,
    # Set up spatial chunking
    "chunks": {'hru': 1},
    # Set up input variable mapping
    "forcing_vars": {"Tmin": "t_min", "Tmax": "t_max", "prcp": "prec", "wind": "wind",},
    "state_vars": {"Tmin": "t_min", "Tmax": "t_max", "prcp": "prec", "wind": "wind",},
    "domain_vars": {"lon": "lon", "lat": "lat", "elev": "elev", "mask": "mask",},
    # Set up output specifications
    "out_vars": {
        'temp': {'out_name': 'airtemp', 'units': 'K'}, 'prec': {'out_name': 'pptrate', 'units': 'mm s-1'},
        'air_pressure': {'out_name': 'airpres', 'units': 'Pa'}, 'shortwave': {'out_name': 'SWRadAtm'},
        'longwave': {'out_name': 'LWRadAtm'}, 'spec_humid': {'out_name': 'spechum' }, 'wind': {'out_name': 'windspd' },
        'vapor_pressure': {'out_name': 'vapor_pressure', 'units': 'hPa'}},
}

# Some helper functions

Additionally we are going to provide some helper functions which will be useful for you to convert the MetSim output to SUMMA input, as well as a function which makes it easier to create new file managers for each of the different forcing datasets. You may simply run this cell to move forward.

In [ ]:
def metsim_to_summa(ms, base_dataset='./data/reynolds/forcing_sheltered.nc'):
    """
    Convert a metsim object's output to summa compatible input
    This simply adds the `data_step` and `hruId` variables from
    the given `base_dataset`. This appends to the metsim dataset.
    
    Parameters
    ----------
    ms: MetSim
        A MetSim object which has had the `run` method called
    base_dataset: str
        The path to a dataset which is used to populate the
        `data_step` and `hruId` variables
        
    Returns
    -------
    The update `xarray.Dataset` from the MetSim run
    """
    base_dataset = xr.open_dataset(base_dataset)
    with ms.open_output() as ds:
        ds['data_step'] = base_dataset['data_step']
        ds['hruId'] = base_dataset['hruId']
        out_ds = ds.load()
    out_prefix = ms.params["out_prefix"]
    out_suffix = ms.get_nc_output_suffix(out_ds["time"].to_series())
    out_filename = f'{out_prefix}_{out_suffix}.nc'
    out_dirname = os.path.abspath(ms.params['out_dir'])
    out_ds.to_netcdf(f'{out_dirname}/{out_filename}')
    return out_ds

    
def create_file_manager(ms, fm, new_suffix):
    """
    Creates a new file manager that points to a forcing
    file from a metsim run.
    
    Parameters
    ----------
    ms: MetSim
        A MetSim object which has had the `run` method called.
    fm: FileManager
        A FileManager to use as a template
    new_suffix: str
        The new suffix for the new file manager
        
    Returns
    -------
    The path to the new file manager
    """
    out_prefix = ms.params["out_prefix"]
    with ms.open_output() as ds:
        out_suffix = ms.get_nc_output_suffix(ds["time"].to_series())
    out_filename = f'{out_prefix}_{out_suffix}.nc'
    out_dirname = os.path.abspath(ms.params['out_dir'])
   
    new_force_file_list = f'summa_zForcingFileList_{new_suffix}.txt'
    new_file_manager = f'summa_fileManager_{new_suffix}.txt'
    with open(f'{fm["settingsPath"].value}/{new_force_file_list}', 'w') as f:
        f.write(f"'{out_filename}'")
    fm['forcingListFile'].value = new_force_file_list
    fm.file_name = new_file_manager
    fm.write()
    return f'./{fm.original_path}/{fm.file_name}'

# Longwave radiation parameterizations

MetSim has a number of different options and parameterizations available for the various processes. Here we will explore the avaialble parameterizations for clear sky longwave radiation. These all follow the same overall parameterization that was presented during the lecture:

### $$ LW = E\sigma T^4 $$

where $\sigma \approx 5.67\cdot 10^{-8} W m^{-2} K^{-4}$ is the Stephan Boltzmann constant, $T$ is temperature (in Kelvin), and $E$ is the emissivity. Each of the various longwave parameterizations thus define their own emissivity function. Here is a brief summary of how they are computed along with the relevant citations:

## TVA
Heat and mass transfer between a water surface and the atmosphere. Tennessee Valley Authority, Norris, TN. Laboratory report no. 14. Water resources research report
no. 0-6803.

### $$ E = 0.74 + 0.0049 \cdot e $$

where $e$ is vapor pressure in hPa

## Anderson
Anderson, E.R., 1954. Energy budget studies, water loss
investigations: lake Hefner studies. U.S. Geol. Surv. Prof. Pap. 269,
71–119 Available from U.S. Geological Survey, 807 National Center, Reston, VA 20192.

### $$ E = 0.68 + 0.036 \cdot \sqrt{e}$$

## Brutsaert
Brutsaert, W., 1975. On a derivable formula for long-wave radiation
from clear skies. Water Resour. Res. 11 (5), 742–744,
doi:10.1029/WR011i005p00742.

### $$ E = 1.24 \cdot (e/T)^{0.143} $$

where $e$ is vapor pressure in hPa and $T$ is temperature in Kelvin

## Satterlund
Satterlund, D.R., 1979. An improved equation for estimating long-wave
radiation from the atmosphere. Water Resour. Res. 15 (6), 1649–1650,
 doi:10.1029/WR015i006p01649.
        
### $$ E = 1.08 (1 - exp(-e^{T/2016} )$$

where $e$ is vapor pressure in hPa and $T$ is temperature in Kelvin

## Idso
Idso, S.B., 1981. A set of equations for full spectrum and 8- to
14-µm and 10.5- to 12.5-µm, thermal radiation from cloudless skies.
Water Resour. Res. 17 (2), 295–304, doi:10.1029/WR017i002p00295.

### $$ E = 0.7 + 5.95\cdot 10^{-5} \cdot e \cdot exp(1500 / T) $$

where $e$ is vapor pressure in hPa and $T$ is temperature in Kelvin

## Prata

Prata, A.J., 1996. A new long-wave formula for estimating downward
clear-sky radiation at the surface. Q. J. R. Meteor. Soc. 122 (533),
1127–1151, doi:10.1002/qj.49712253306.

### $$ r = (46.4 \cdot (e/T) $$
### $$ E = 1- (1+r) \cdot exp\left(-\sqrt{1.2 + 3\cdot r)}\right) $$

where $e$ is vapor pressure in hPa and $T$ is temperature in Kelvin

# Running MetSim and preparing to run SUMMA

Each of the methods outlined above are able to be run by setting the `config['lw_type']` to the corresponding option given in the `lw_options` list outlined below. 

Using your knowledge of MetSim:
* Run each of the longwave parameterizations with MetSim (don't forget to change `config['out_prefix']`, so you don't overwrite output!)
* Convert the MetSim output to SUMMA input
* Save a new file manager with the `create_file_manager` function (use the longwave option as the new suffix)

If you're having trouble getting this set up see the `metsim_basics` notebook for an example.

In [ ]:
lw_options = ['tva', 'anderson', 'brutsaert', 'idso', 'satterlund', 'prata']

# In a loop you can add to these by running the `.append` method - for help search `python append to list`
new_file_managers = []
metsim_outputs = []
base_file_manager = ps.FileManager('./settings/reynolds', 'summa_fileManager.txt')

for lw_opt in lw_options:
    print(f'Running {lw_opt} now!')
    # Todo:
    # 1. Update the `config['lw_type']` to the new longwave option
    # 2. Update the `config['out_prefix']` to specify the new output file
    # 3. Instantiate and run metsim given the updated configuration
    # 4. Convert the metsim output to summa format, and append it to the 
    #    `metsim_outputs` list
    # 5. Save a new file manager using the `create_file_manager` function,  
    #    and append it to the `new_file_managers` list.

# Plotting the longwave parameterizations
Now, using the metsim outputs you've generated let's take a moment to see how the different parameterizations look. Make two plots/sets of plots that show:
* Simple timeseries of the estimated longwave for each parameterization
* Scatter plots of the estimated longwave against the vapor pressure for each parameterization

# Running SUMMA and comparing results

Now that you've generated six different forcing datasets with different longwave radiation you should be able to run each of them through SUMMA. To do so, we've set you up with a little bit of boilerplate that defines the ensemble of simulations, runs them, and merges the output into a single `xarray.Dataset`. Using your knowledge of running and manipulating SUMMA output:
* Run the SUMMA ensemble
* Plot the SWE for each of the longwave parameterizations
* Explore how these longwave parameterizations affect the other variables - what about snow density or surface temperature?

In [ ]:
assert len(new_file_managers) == 6, ('The previous step is still not complete! Please go back',
                                     ' and fill lin the loop from the previous code block to continue')

ens_config = {lw_opt: {'file_manager': nfm}
              for lw_opt, nfm in zip(lw_options, new_file_managers)}

summa_executable = 'summa.exe'
file_manager = './settings/reynolds/summa_fileManager.txt'
lw_ens = ps.Ensemble(summa_executable, ens_config, num_workers=3)

lw_ens.run('local')
summary = lw_ens.summary()
print(summary)

In [ ]:
obs = xr.open_dataset('./data/reynolds/ReynoldsCreek_valData.nc')
obs = obs.sel(time=slice('2005/07/01', '2006/09/30'))
ds_list = [s.output.load().isel(hru=0, gru=0) for s in lw_ens.simulations.values()]
lw_ds = xr.concat(ds_list, dim='lw_type')
lw_ds['lw_type'] = lw_options